In [2]:
%load_ext autoreload
%autoreload

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from mirisim.skysim import Background, sed, Point, Skycube
from mirisim.config_parser import SimConfig, SimulatorConfig, SceneConfig
from mirisim import MiriSimulation

from importlib import reload

In this notebook we will demonstrate how to generate simulated MIRI time series LRS (slitless) data. To install Mirisim, see the [the public release webpage](http://miri.ster.kuleuven.be/bin/view/Public/MIRISim_Public). The target for the mock observations is WASP-62, an exoplanet host star with the following properties from [the exoplanet encyclopaedia](http://exoplanet.eu/catalog/wasp-62_b/):

* spectral type F7
* T_bb = 6230 K
* K = 8.994

K magnitude of 8.994 corresponds to a flux of 20e3 microJy at 8 micron (value obtained from the ETC), and we will use this value for the source generation.

Using the ETC, I calculated the following number of groups for a high-SNR but unsaturated image:
ngrps = 100

We will perform the simulation with 20 integrations. This corresponds to a total exposure time of 0.159 s * 100 * 20  = 318 seconds or 5.3 minutes. **NOTE** MIRISim cannot simulated a changing flux within an expsoure, so there is no actual "transit" or "eclipse" signature in the data. 

### Steps in setting up the simulation

This notebook will go through the following steps:

* Create the scene
* Set up the simulation
* Run the simulation

Each step has its own function, which we will define at the top of the notebook. 

## Step 0: Definition of some helper functions

We will define 2 helper functions:

1. star_bb_scene: defines the "input scene", i.e the star and the background. it assumes the star is a black body and takes the BB parameters as input
2. lrstso_sim_confg: sets up the simulation with appropriate parameters



In [21]:
def star_bb_scene(scene_file=None, params={'Temp': None, 'wref': None, 'flux': None}, background=None, out=True):
    # place a star object in the centre of the field

    if scene_file is not None:
        print('Creating scene. Scene will be written to file {0}'.format(scene_file))
    else:
        print('No filename provided. Scene will be written out to file scene.ini')
        scene_file = 'scene.ini'

    star = Point(Cen=(0,0))
    bbparams = params
    bb = sed.BBSed(**bbparams)

    # set the star's SED to this blackbody SED
    star.set_SED(bb)

    # now we can add a background if requested:
    bg = Background(level=background, gradient=0., pa=0.)
    scene = star + bg    

    # now we also want to create the ini file
    targetlist = [star]

    ## export to ini file
    scene_config = SceneConfig.makeScene(loglevel=0,
                                        background=bg,
                                        targets = targetlist)
    
    if out:
        if os.path.exists(scene_file):
            print('File {0} already exists. Will be overwritten.'.format(scene_file))
        scene_config.write(scene_file)
        
    
    return scene_config


In [24]:
def lrstso_sim_config(mode='lrs', arr='SLITLESSPRISM', ngrp=None, nint=None, nexp=1, webbpsf=True, scene_file=None, out=True):
    
    '''
    Function to set up the imaging TSO simulation. Arguments:
    - mode: the MIRI mode to use (default='lrs') [string]
    - arr: array configureation (default='SLITLESSPRISM') [string]
    - ngrp: the number of groups (minimum of 2 required; minimum of 5 is recommended) [integer]
    - nint: number of integrations [integer]
    - nexp: number of exposures (default = 1, as recommended for TSOs) [integer]
    - scene_file: scene filename, as returned by the scene generation function [string]
    - out: set to True if you want to write the sim configuration out to file (default=True)[boolean]
    
    
    Notes on other simulation parameters, as we are only using this function for imaging simulations:
    - POP: always 'IMA'
    - parameters disperser, detector and mrs_* are included in teh list but NOT ACCESSED
    - ima_mode: always use FAST mode for TSOs
    
    '''
    
    if (mode == 'lrs') & (arr == 'SLITLESSPRISM'):
        op_path = 'IMA'
        cfg = 'LRS_SLITLESS'
        filt = 'P750L'
    
    else:
        raise ValueError("This mode is not supported in this function.")
    
    if arr not in ['SLITLESSPRISM']:
        raise ValueError("Array configuration not supported for LRS slitless TSOs.")
    
    if os.path.exists(scene_file):
        print("Found scene file {0}".format(scene_file))
    else:
        raise ValueError('Scene file not found.')

    if ngrp < 2:
        raise ValueError("Number of groups must be 2 or larger.")
    
    fbase = scene_file.split('_')[0]
    simname = '{3}_{4}_{0}G{1}I{2}E'.format(ngrp, nint, nexp, fbase, arr)
    
    sim_config = SimConfig.makeSim(
        name = simname,    # name given to simulation
        scene = scene_file, # name of scene file to input
        rel_obsdate = 0.0,          # relative observation date (0 = launch, 1 = end of 5 yrs)
        POP = op_path,                # Component on which to center (Imager or MRS)
        ConfigPath = cfg,  # Configure the Optical path (MRS sub-band)
        Dither = False,             # Don't Dither
        StartInd = 1,               # start index for dither pattern [NOT USED HERE]
        NDither = 2,                # number of dither positions [NOT USED HERE]
        DitherPat = 'lrs_recommended_dither.dat', # dither pattern to use [NOT USED HERE]
        disperser = 'SHORT',        # [NOT USED HERE]
        detector = 'SW',            # [NOT USED HERE]
        mrs_mode = 'SLOW',          # [NOT USED HERE]
        mrs_exposures = 10,          # [NOT USED HERE]
        mrs_integrations = 3,       # [NOT USED HERE]
        mrs_frames = 5,             # [NOT USED HERE]
        ima_exposures = nexp,          # number of exposures
        ima_integrations = nint,       # number of integrations
        ima_frames = ngrp,             # number of groups (for MIRI, # Groups = # Frames)
        ima_mode = 'FAST',          # Imager read mode (default is FAST ~ 2.3 s)
        filter = filt,          # Imager Filter to use
        readDetect = arr         # Portion of detector to read out,
    )


    # write the simulation config out to file, if out was set to True. the output filename is the simname, followed b the number of groups, ints and exp, for easy reference.
    simout = '{0}_simconfig.ini'.format(simname)
    if out:
        sim_config.write(simout)

    return sim_config
    


## Step 1: Creating the input scene (star_scene)

Here we'll create the input scene for the simulations using the function star_scene(). Arguments:

* scene_file: the filename for the .ini file
* write_cube: write the scene image out to a FITS file (optional; default=False)

The function returns a mirisim.skysim.scenes.CompositeSkyScene object.


In [25]:
star_temp = 6230.
bb_wref = 2.0
star_flux = 20e3

scene_cfg = star_bb_scene(scene_file='lrstso_star_scene.ini', params={'Temp': star_temp, 'wref': bb_wref, 'flux': star_flux}, background='low')

2021-05-20 14:19:23,991 - INFO - Initializing Point
2021-05-20 14:19:23,992 - INFO - Initializing Background


Creating scene. Scene will be written to file lrstso_star_scene.ini
File lrstso_star_scene.ini already exists. Will be overwritten.
lrstso_star_scene.ini exists, overwrite (y/[n])?y


## Step 2: Configuring the simulation

Now we'll set up the simulations and prepare to run them. Before we write out a new simconfig file, we delete any existing files in the directory. I'll set it up to loop through the different simulations. For this we use the function ``lrstso_sim_config`` defined above. Check the docstring for descriptions and default values of the arguments. 

The function will write out another .ini file containing the simulation configuration, and it returns the output filename for further use.

In [26]:
old_files = glob.glob('*_simconfig.ini')

for of in old_files:
    os.remove(of)
    
ngrp = 100
nint = 20

sim_cfg = lrstso_sim_config(mode='lrs', arr='SLITLESSPRISM', ngrp=ngrp, nint=nint, nexp=1, 
                   scene_file=scene_ini, out=True)   

print(sim_ini)

Found scene file lrstso_star_scene.ini
lrstso_SLITLESSPRISM_100G20I1E_simconfig.ini


### Step 3: Run the simulation

In the following step we'll run the simulation using 3 input files:the scene, the simulation configuration, and the simulator setup file. We make one modification to the "default" simulator configuration, and that is to use the WebbPSF model. This PSF model is more accurate than the default instrument model, as it includes the telescope model. 

After the simulation has run, the code renames the output directory to include the simulation settings to the directory.


In [ ]:
#configure the simulator engine - this requires no editing from the default
simulator_config = SimulatorConfig.makeSimulator(take_webbPsf=True)

cfg_file = glob.glob('*simconfig.ini')
tmp = cfg_file[0].split('.')
fcomps = tmp[0].split('_')

sim = MiriSimulation(sim_config=sim_cfg, scene_config=scene_cfg,
                        simulator_config=simulator_config, loglevel='WARNING', local=False)
sim.run()




You may want to rename the output folder on completion. 

-- END